In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.table import Table
from astropy import table
from astroquery.gaia import Gaia
from scipy.optimize import minimize
import stam
from tqdm import tqdm
%matplotlib tk

In [2]:
PARSEC_path = '../data/PARSECv2.0/w_i=0.6,lin_age/'
models = stam.getmodels.read_parsec(path=PARSEC_path)

Taking PARSEC files from ../data/PARSECv2.0/w_i=0.6,lin_age/


In [3]:
clusters = Table.read('../data/hunt_clusters/clusters.csv')
members = Table.read('../data/hunt_clusters/members.csv')
sources = Table.read('../table_best.fits')


# sources_done = Table.read('../table.fits')
# sources = Table.read('../table_B.fits')
# cut = ~np.isin(sources['source_id'], sources_done['source_id']) & (sources['cst'] > 5) & (sources['class_50'] > 0.5) & (sources['probability'] > 0.99)
# sources = sources[cut]
# sources['idx'] = 29 + np.arange(len(sources))

In [4]:
## Define the (empirical) model color-color relation to be compared with the data
## The fit will determine the best value of E(B-V) (at B-V=0) to minimize the chi^2

data = pd.read_csv('../data/other/color_relations.csv')

eta = lambda bv0: 0.97 - 0.09 * bv0

e_bv = lambda bv0,e_bv0: eta(bv0)/eta(0) * e_bv0 

e_ub = lambda e_bv,alpha,beta,gamma: alpha * e_bv + beta * e_bv**2 + gamma * e_bv**3

def redden_f70(e_bv0):
    bv0_arr = data['(B-V)0'].values
    ub0_arr = data['(U-B)0'].values
    alpha_arr = data['alpha'].values
    beta_arr = data['beta'].values
    gamma_arr = data['gamma'].values
    
    e_bv_arr = e_bv(bv0_arr,e_bv0)
    e_ub_arr = e_ub(e_bv_arr,alpha_arr,beta_arr,gamma_arr)

    bv0_arr_reddened = bv0_arr + e_bv_arr
    ub0_arr_reddened = ub0_arr + e_ub_arr
    
    return bv0_arr_reddened,ub0_arr_reddened

def chi_2_ccd(e_bv0,bv_arr,ub_arr):
    bv0_arr_reddened,ub0_arr_reddened = redden_f70(e_bv0)
    
    cut = (bv_arr > bv0_arr_reddened[0]) & (bv_arr < bv0_arr_reddened[-1])
    bv_arr = bv_arr[cut]
    ub_arr = ub_arr[cut]

    ub0_f70 = np.interp(bv_arr,bv0_arr_reddened,ub0_arr_reddened)
    chi_2 = np.sum((ub_arr - ub0_f70)**2)
    return chi_2

def f70_lims(e_bv0):
    bv_f70,ub_f70 = redden_f70(e_bv0)
    up_lim = ub_f70 + 0.017
    low_lim = np.interp(bv_f70-0.020,bv_f70,ub_f70)
    return up_lim,low_lim

def in_f70_lims(tbl,bv_f70,up_lim,low_lim):
    up_lim = np.interp(tbl['bv'],bv_f70,up_lim)
    low_lim = np.interp(tbl['bv'],bv_f70,low_lim)
    cut = (tbl['ub'] < up_lim) & (tbl['ub'] > low_lim) & (tbl['bv'] < bv_f70[-1]) & (tbl['bv'] > bv_f70[0])
    return tbl[cut]

def fit_and_plot(idx):
    clstr_id = sources[sources['idx']== idx]['id'][0]
    clstr = clusters[clusters['id']==clstr_id]
    memb = members[members['id']==clstr_id]

    cut = (memb['probability'] > 0.99) & (memb['non_single_star'] == 0)
    memb = memb[cut]

    query = f'''SELECT source_id, u_jkc_mag, v_jkc_mag, b_jkc_mag FROM gaiadr3.synthetic_photometry_gspc WHERE source_id IN {tuple(memb['source_id'])}'''
    job = Gaia.launch_job(query)
    r = job.get_results()

    cut = (~r['u_jkc_mag'].mask) & (~r['v_jkc_mag'].mask) & (~r['b_jkc_mag'].mask)
    r = r[cut]

    tbl = table.join(memb,r,keys='source_id')
    tbl['mg'] = tbl['phot_g_mean_mag'] - 5 * np.log10(1000/tbl['parallax']) + 5
    tbl['bv'] = tbl['b_jkc_mag'] - tbl['v_jkc_mag']
    tbl['ub'] = tbl['u_jkc_mag'] - tbl['b_jkc_mag']
    tbl = tbl['source_id','mg','bp_rp','bv','ub']

    ebv_guess = clstr['a_v_50'][0] / 3.1
    tbl.sort('bv')

    res = minimize(chi_2_ccd,ebv_guess,args=(tbl['bv'],tbl['ub']),bounds=[(0,5*ebv_guess)])
    ebv = res.x[0]

    # fig,(ax1,ax2) = plt.subplots(nrows=2,dpi=120,gridspec_kw={'height_ratios':[3,1]})
    # fig.tight_layout()
    bv_f70,ub_f70 = redden_f70(ebv)
    cut = tbl['bv'] < bv_f70[-1]
    bv_plot = tbl['bv'][cut]
    ub_plot = tbl['ub'][cut]

    up_lim,low_lim = f70_lims(ebv)
    tbl_mtso = in_f70_lims(tbl,bv_f70,up_lim,low_lim)

    # ax1.plot(bv_f70,ub_f70,'k-',label=f'F70 ebv={ebv:.3f}')
    # ax1.fill_between(bv_f70,up_lim,low_lim,alpha=0.5,color='Gold')
    # ax1.scatter(bv_plot,ub_plot,s=10,c='r',label='Data')
    # ax1.scatter(tbl_mtso['bv'],tbl_mtso['ub'],s=1,c='g',label='Good MSTO')
    # ax1.set_xlabel('B-V')
    # ax1.set_ylabel('U-B')
    # ax1.invert_yaxis()
    # ax1.legend()
    # ax1.set_title(f'candidate {idx}')
    # textstr = f'{len(memb)} reliable cluster members\n {len(tbl_mtso)} good MSTO stars'
    # props = dict(boxstyle='round', facecolor='k', alpha=0.5)
    # ax1.text(0.05, 0.2, textstr, transform=ax1.transAxes, fontsize=12,
    #         verticalalignment='top', bbox=props)
    # ax2.set_xlabel('U-B')
    # ax2.set_ylabel(r'$\Delta$ (U-B)')
    # ax2.scatter(ub_plot,ub_plot-np.interp(bv_plot,bv_f70,ub_f70),s=10,c='r')
    # ax2.scatter(tbl_mtso['ub'],tbl_mtso['ub']-np.interp(tbl_mtso['bv'],bv_f70,ub_f70),s=1,c='g')
    # ax2.fill_between(ub_f70,up_lim-ub_f70,low_lim-ub_f70,alpha=0.5,color='Gold')
    # ax2.invert_yaxis()
    return ebv,tbl_mtso

# color_fil_1, color_fil_2, mag_fil = "G_BP", "G_RP", "G" ## no rotation
# model = models ## no rotation

color_fil_1, color_fil_2, mag_fil = "G_BP_i45", "G_RP_i45", "G_i45" ## rotation with 45 deg inclination

def chi_2_cmd(age,mh,bp_rp,mg,ebv):
    if age <= 45:
            age_res = 5e-4 # [Gyr]
    elif age <= 100:
        age_res = 1e-3
    else:
        age_res = 1e-2
    stage_min = 0  # pre-main sequence
    stage_max = 10  # red giant branch
    mass_min = 0  # [Msun]
    mass_max = 20  # [Msun]
    tracks = stam.gentracks.get_isotrack(models, [age * 1e-3, mh], params=("age", "mh"), return_table=True,
                                    age_res=age_res, mass_min=mass_min, mass_max=mass_max, mh_res = 0.02,
                                    stage=None, stage_min=stage_min, stage_max=stage_max, sort_by="age", color_filter1=color_fil_1, color_filter2=color_fil_2,
                mag_filter=mag_fil)
    x = np.array(tracks["bp_rp"])
    y = np.array(tracks["mg"])
    xx,yy = [a for _,a in sorted(zip(y,x))], np.sort(y)
    
    e_bprp, A_G = stam.gaia.get_extinction_in_band(ebv,mag_filter="G",color_filter1='G_BP',color_filter2='G_RP')
    bp_rp_corrected = bp_rp - e_bprp
    mg_corrected = mg - A_G
    bp_rp_model = np.interp(mg_corrected,yy,xx)
    chi_2 = np.sum((bp_rp_corrected - bp_rp_model)**2)
    return chi_2/len(bp_rp_corrected)

def fit_cluster_cmd(idx):
    ebv,tbl_mtso = fit_and_plot(idx)
    if len(tbl_mtso) >= 4:
        age = np.concatenate([np.arange(10,45,1),np.arange(45,100,5),np.arange(100,1001,50)])
        mh = np.arange(-0.6,0.07,0.05)
        age,mh = np.meshgrid(age,mh)
        chi2_mat = np.zeros((mh.shape[0],age.shape[1]))
        for i in tqdm(range(age.shape[1])):
            for j in range(mh.shape[0]):
                chi2_mat[j,i] = chi_2_cmd(age[j,i],mh[j,i],tbl_mtso['bp_rp'],tbl_mtso['mg'],ebv)

        j,i = np.unravel_index(np.argmin(chi2_mat), chi2_mat.shape)
        age_best = age[j,i]
        mh_best = mh[j,i]

        if age_best <= 45:
            age_res = 5e-4 # [Gyr]
        elif age_best <= 100:
            age_res = 1e-3
        else:
            age_res = 1e-2
        stage_min = 0  # pre-main sequence
        stage_max = 10  # red giant branch
        mass_min = 0  # [Msun]
        mass_max = 10  # [Msun]

        tracks = stam.gentracks.get_isotrack(models, [age_best * 1e-3, mh_best], params=("age", "mh"), return_table=True,
                                            age_res=age_res, mass_min=mass_min, mass_max=mass_max, mh_res = 0.02,
                                            stage=None, stage_min=stage_min, stage_max=stage_max, sort_by="age", color_filter1=color_fil_1, color_filter2=color_fil_2,
                        mag_filter=mag_fil)

        x = np.array(tracks["bp_rp"])
        y = np.array(tracks["mg"])
        xx,yy = [a for _,a in sorted(zip(y,x))], np.sort(y)

        e_bprp, A_G = stam.gaia.get_extinction_in_band(ebv,mag_filter="G",color_filter1='G_BP',color_filter2='G_RP')
        bp_rp_corrected = tbl_mtso['bp_rp'] - e_bprp
        mg_corrected = tbl_mtso['mg'] - A_G
        bp_rp_model = np.interp(np.union1d(mg_corrected,yy),yy,xx)

        clstr_id = sources[sources['idx']== idx]['id'][0]
        memb = members[members['id']==clstr_id]
        cut = (memb['probability'] > 0.99) & (memb['non_single_star'] == 0)
        memb = memb[cut]
        memb['mg'] = memb['phot_g_mean_mag'] - 5 * np.log10(1000/memb['parallax']) + 5

        fig,ax = plt.subplots(dpi=120)
        ax.scatter(bp_rp_corrected,mg_corrected,s=10,c='r',label='Good MSTO')
        ax.scatter(memb['bp_rp'] - e_bprp,memb['mg'] - A_G,s=5,c='b',label='All members')
        ax.plot(bp_rp_model,np.union1d(mg_corrected,yy), 'ko', markersize=1,label=f'PARSEC age={int(age_best)} Myr, [Fe/H]={mh_best:.2f}')
        ax.set_xlabel('BP-RP')
        ax.set_ylabel('G')
        ax.invert_yaxis()
        ax.legend()
        ax.set_title(f'candidate {idx}')
        plt.show()
        return age_best, mh_best, ebv
    else:
        print(f'Candidate {idx} has only {len(tbl_mtso)} good MSTO stars, not enough to fit the isochrone.')
        return np.nan, np.nan, np.nan
    
def photometric_mass(age,mh,ebv,src):    
    src['e_bv'] = ebv
    color_excess_key = 'e_bv'
    n_realizations = 10000
    correct_extinction = True
    if age <= 45:
        age_res = 5e-4 # [Gyr]
    elif age <= 100:
        age_res = 1e-3
    else:
        age_res = 1e-2
    stage_min = 0  # pre-main sequence
    stage_max = 10  # red giant branch
    mass_min = 0  # [Msun]
    mass_max = 10  # [Msun]
        
    m1, m1_err = stam.run.multirun(src, vals=[age * 1e-3, mh], params=("age", "mh"), suffix="", is_save=False,
                                                track_type="isotrack", assign_param="mass", is_extrapolate=False, rbf_func="linear",
                                                    output_type="csv", output_path="./stam_output/", n_realizations=n_realizations, interp_fun="griddata",
                                                    models=models, correct_extinction=correct_extinction, color_excess_key=color_excess_key , mh_res = 0.02,
                                                    use_reddening_key=False, mass_min=mass_min, mass_max=mass_max,stage=None, stage_min=stage_min, stage_max=stage_max, age_res=age_res,
                                                    color_filter1=color_fil_1, color_filter2=color_fil_2, mag_filter=mag_fil)
    return m1[0], m1_err[0]
    

In [5]:
new_sources = sources.copy()
m1_col = []
m1_err_col = []
age_col = []
mh_col = []
av_col = []
for idx in sources['idx']:
    age,mh,ebv = fit_cluster_cmd(idx)
    if np.isnan(age):
        new_sources.remove_row(np.where(new_sources['idx'] == idx)[0][0])
    else:
        m1,m1_err = photometric_mass(age,mh,ebv,sources[sources['idx'] == idx])
        m1_col.append(m1)
        m1_err_col.append(m1_err)
        age_col.append(age)
        mh_col.append(mh)
        av_col.append(ebv * 3.1)

100%|██████████| 65/65 [00:19<00:00,  3.27it/s]
2024-03-12 10:32:40 - INFO - Using isotrack...
2024-03-12 10:32:40 - INFO - Calculating Gaia uncertainties...
2024-03-12 10:32:40 - INFO - Applying extinction correction...
2024-03-12 10:32:40 - INFO - Assigning mass using 10000 realizations...
2024-03-12 10:32:40 - INFO - Assigning mass...
2024-03-12 10:32:40 - INFO - Ignoring twin binary sequence...


Using griddata interpolation...


100%|██████████| 65/65 [00:19<00:00,  3.27it/s]
2024-03-12 10:33:00 - INFO - Using isotrack...
2024-03-12 10:33:00 - INFO - Calculating Gaia uncertainties...
2024-03-12 10:33:00 - INFO - Applying extinction correction...
2024-03-12 10:33:00 - INFO - Assigning mass using 10000 realizations...
2024-03-12 10:33:00 - INFO - Assigning mass...
2024-03-12 10:33:00 - INFO - Ignoring twin binary sequence...


Using griddata interpolation...


100%|██████████| 65/65 [00:19<00:00,  3.26it/s]
2024-03-12 10:33:20 - INFO - Using isotrack...
2024-03-12 10:33:20 - INFO - Calculating Gaia uncertainties...
2024-03-12 10:33:20 - INFO - Applying extinction correction...
2024-03-12 10:33:20 - INFO - Assigning mass using 10000 realizations...
2024-03-12 10:33:20 - INFO - Assigning mass...
2024-03-12 10:33:20 - INFO - Ignoring twin binary sequence...


...

100%|██████████| 65/65 [00:19<00:00,  3.26it/s]
2024-03-12 10:34:42 - INFO - Using isotrack...
2024-03-12 10:34:42 - INFO - Calculating Gaia uncertainties...
2024-03-12 10:34:42 - INFO - Applying extinction correction...
2024-03-12 10:34:42 - INFO - Assigning mass using 10000 realizations...
2024-03-12 10:34:42 - INFO - Assigning mass...
2024-03-12 10:34:42 - INFO - Ignoring twin binary sequence...


Using griddata interpolation...


100%|██████████| 65/65 [00:19<00:00,  3.27it/s]
2024-03-12 10:35:02 - INFO - Using isotrack...
2024-03-12 10:35:02 - INFO - Calculating Gaia uncertainties...
2024-03-12 10:35:02 - INFO - Applying extinction correction...
2024-03-12 10:35:02 - INFO - Assigning mass using 10000 realizations...
2024-03-12 10:35:02 - INFO - Assigning mass...
2024-03-12 10:35:02 - INFO - Ignoring twin binary sequence...


Using griddata interpolation...


100%|██████████| 1/1 [00:00<00:00, 423.28it/s]


In [6]:
new_sources['m1'] = m1_col
new_sources['m1_err'] = m1_err_col
new_sources['age'] = age_col
new_sources['mh_for_mass_interp'] = mh_col
new_sources['av_for_mass_interp'] = av_col
new_sources.write('../table_best.fits',overwrite=True)